In [ ]:
!pip install easyocr

In [ ]:
#@title Imports
from transformers import AutoImageProcessor, AutoModelForObjectDetection
import torch
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import easyocr
import cv2
import numpy as np

In [ ]:
#@title Loading the Detection Transformer
device = "cuda" if torch.cuda.is_available() else "cpu"
# 2. Load Model and Processor
model_id = "justjuu/rtdetr-v2-license-plate-detection"
processor = AutoImageProcessor.from_pretrained(model_id, use_fast = True)
model = AutoModelForObjectDetection.from_pretrained(model_id)
model = model.to(device)
model.eval()

In [ ]:
#@title Loading the OCR
reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())

In [ ]:
#@title Function to extract License Plate Number
import easyocr
import cv2
import numpy as np

def extract_license_plate_text(image, box):

    x1, y1, x2, y2 = map(int, box.tolist())

    # Get image dimensions to avoid cropping outside the image
    img_w, img_h = image.size

    # --- IMPROVEMENT: Add 5% Padding ---
    pad_w = (x2 - x1) * 0.05
    pad_h = (y2 - y1) * 0.05

    # Apply padding and ensure we don't go < 0 or > image size (Clamping)
    crop_x1 = max(0, int(x1 - pad_w))
    crop_y1 = max(0, int(y1 - pad_h))
    crop_x2 = min(img_w, int(x2 + pad_w))
    crop_y2 = min(img_h, int(y2 + pad_h))

    # Crop with padding
    plate_crop = image.crop((crop_x1, crop_y1, crop_x2, crop_y2))

    # Convert PIL Image to numpy array for EasyOCR
    plate_crop_np = np.array(plate_crop)

    # 1. OCR Inference
    # allowlist: restricts detection to Uppercase A-Z and 0-9 (perfect for plates)
    results = reader.readtext(
        plate_crop_np,
        allowlist='ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 ',
        canvas_size=512,
    )

    # 2. Extract best result
    # EasyOCR returns a list of tuples: (bbox, text, confidence)
    if not results:
        return None
    elif len(results) == 1:
        return results[0][1]
    else:
      word = []
      for result in results:
        word.append(result[1])
      return " ".join(word)

In [ ]:
#@title Function to Plot the Bounding Box along with License Plate Number
def plot_predictions_with_ocr(results, images):
    for result, image in zip(results, images):
        fig, ax = plt.subplots(1, figsize=(8, 8))
        ax.imshow(image)
        ax.axis("off")

        for score, label, box in zip(
            result["scores"], result["labels"], result["boxes"]
        ):
            # Draw bounding box
            x1, y1, x2, y2 = box.tolist()
            w, h = x2 - x1, y2 - y1

            rect = patches.Rectangle(
                (x1, y1), w, h,
                linewidth=2,
                edgecolor="green",
                facecolor="none"
            )
            ax.add_patch(rect)
            # 🔥 OCR extraction
            plate_text = extract_license_plate_text(image, box)

            # Display label + OCR text
            caption = f"{plate_text} ({score:.2f})"

            ax.text(
                x1,
                y1 - 8,
                caption,
                color="white",
                fontsize=10,
                bbox=dict(facecolor="black", alpha=0.6, pad=2)
            )

In [ ]:
#@title Inference Pipeline
image = Image.open("./Screenshot (192).png").convert("RGB")

# Pre processing
inputs = processor(images=image, return_tensors="pt")
# Move inputs to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Inference
with torch.no_grad():
    outputs = model(**inputs)

# Post-process (Non-Maximum Suppression is effectively handled by the DETR architecture, but we define thresholds)
target_sizes = torch.tensor([image.size[::-1]]).to(device)
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.5)

plot_predictions_with_ocr(results, [image])